## Library

In [21]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

from collections import Counter
# Lemmatize with POS Tag
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn import linear_model


## Loading Data 

In [22]:
D = np.load('tf_idf_vector_matrix.npy') # load

In [23]:
D.shape

(9025, 22336)

In [24]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [25]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 27 columns):
index                    9025 non-null int64
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status               

In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99810 entries, 0 to 99809
Data columns (total 5 columns):
userId       99810 non-null int64
id           99810 non-null float64
rating       99810 non-null float64
movieId      99810 non-null int64
timestamp    99810 non-null int64
dtypes: float64(2), int64(3)
memory usage: 3.8 MB


In [34]:
ratings

,userId,id,rating,movieId,timestamp
0,1,152.0,2.5,1371,1260759135
1,1,9909.0,2.5,31,1260759144
2,1,847.0,2.0,2193,1260759198
3,1,9426.0,2.5,2455,1260759113
4,1,6114.0,3.5,1339,1260759125
...,...,...,...,...,...
99805,671,786.0,2.0,3897,1063503718
99806,671,7443.0,4.0,3751,1065111939
99807,671,1891.0,5.0,1196,1064890635
99808,671,279.0,4.0,1225,1065149143


In [27]:
train.shape

(79848, 5)

In [28]:
test.shape

(19962, 5)

In [47]:
n_users = len(ratings['userId'].unique().astype('int'))
n_items = len(ratings['id'].unique().astype('int'))
n_users

671

In [70]:
len(train[train['userId'] == 1])

18

In [71]:
len(test[test['userId'] == 1])

2

In [31]:
ratings_train = train.values
ratings_test = test.values

In [32]:
ratings_train

array([[1.00000000e+00, 1.05100000e+03, 4.00000000e+00, 1.95300000e+03,
        1.26075919e+09],
       [1.00000000e+00, 1.10300000e+03, 2.00000000e+00, 1.12900000e+03,
        1.26075918e+09],
       [1.00000000e+00, 1.17780000e+04, 2.00000000e+00, 1.26300000e+03,
        1.26075915e+09],
       ...,
       [6.71000000e+02, 8.57000000e+02, 4.00000000e+00, 2.02800000e+03,
        1.06489158e+09],
       [6.71000000e+02, 1.01620000e+04, 4.00000000e+00, 2.35900000e+03,
        1.06350393e+09],
       [6.71000000e+02, 7.11000000e+02, 3.50000000e+00, 4.01900000e+03,
        1.06511196e+09]])

In [38]:
def find_movie_index(list_tmdb_id):
    movie_indexs = []
    for i in list_tmdb_id:
            movie_index = smd[smd['id'] == i].first_valid_index()
            movie_indexs.append(movie_index)
    return movie_indexs

In [36]:
ratings_train

array([[1.00000000e+00, 1.05100000e+03, 4.00000000e+00, 1.95300000e+03,
        1.26075919e+09],
       [1.00000000e+00, 1.10300000e+03, 2.00000000e+00, 1.12900000e+03,
        1.26075918e+09],
       [1.00000000e+00, 1.17780000e+04, 2.00000000e+00, 1.26300000e+03,
        1.26075915e+09],
       ...,
       [6.71000000e+02, 8.57000000e+02, 4.00000000e+00, 2.02800000e+03,
        1.06489158e+09],
       [6.71000000e+02, 1.01620000e+04, 4.00000000e+00, 2.35900000e+03,
        1.06350393e+09],
       [6.71000000e+02, 7.11000000e+02, 3.50000000e+00, 4.01900000e+03,
        1.06511196e+09]])

In [40]:
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0] # get all the value in user_id column
    # need to plus 1 to user_id because after this we will 
    #start with 0 in for-loop
    user_id = user_id + 1
    # get list of user_id index in y which value equal input 
    # user_id
    ids = np.where(y == user_id)[0]
    # get list tmdb_id matching user_id 
    list_tmdb_id = rate_matrix[ids, 1]
    item_ids = find_movie_index(list_tmdb_id)
    
    scores = rate_matrix[ids,2]
    return (item_ids, scores)

In [76]:
d = D.shape[1] # movie vector dimention
W = np.zeros((d, n_users)) 
b = np.zeros((1,n_users))

(1, 671)

In [ ]:
for i in range(n_users):
        idx, scores = get_items_rated_by_user(ratings_train, i)
        clf = Ridge(alpha=0.01, fit_intercept = True)
        idx = np.asarray(idx)
        Xhat = D[idx,:]
        clf.fit(Xhat, scores)
        W[:, i] = clf.coef_
        b[0, i] = clf.intercept_

In [ ]:
Yhat = D.dot(W) + b 
Yhat

In [ ]:
ids, scores = get_items_rated_by_user(ratings_test, 3)
ids = list(ids)
len(ids)

In [ ]:
Yhat.shape

In [ ]:
for i in ids:
    print(Yhat[i,0])

In [ ]:
for i in scores:
    print(i)

In [ ]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_user):
        ids, score_truth = get_items_rated_by_user(rates, n)
        